In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional, Callable

In [3]:
tokenizer = AutoTokenizer.from_pretrained("roneneldan/TinyStories-33M")
model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-33M")

[2023-06-18 19:30:02,121] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-06-18 19:30:04.327277: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [6]:
model = model.to('cuda')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
prompt = "tell me a story"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda')

with torch.no_grad():
  output = model.generate(input_ids, max_length = 1000, num_beams=1)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tell me a story about a princess and a dragon. Do you want to hear a story about a princess and a dragon?"

Lily nodded. She liked stories. She liked princesses and dragons. She liked dragons and dragons. She said, "Yes, please. Tell me a story about a princess and a dragon."

Her mom smiled. She said, "Okay. Let's go to the couch and I'll tell you a story about a princess and a dragon."

Lily and her mom went to the couch. They cuddled and listened. Lily told her mom a story about a princess and a dragon. Her mom listened and nodded. She said, "That's a good story. I like stories about princesses and dragons. They are beautiful and brave. Do you want to hear another one?"

Lily said, "Yes, please. I want to hear another one."

Her mom said, "Okay. Let's hear another one."

Lily and her mom cuddled and listened. They both felt happy and calm. They loved each other and their mom. They loved each other and their stories. They had a good day.



In [13]:
import trlx
from trlx.data.default_configs import TRLConfig, default_ppo_config


In [18]:
default_config = default_ppo_config().to_dict()
#default_config['train']['tracker'] = None
#default_config['train']['batch_size'] = 16
#default_config['train']['epochs'] = 10
default_config['model']['model_path'] = 'roneneldan/TinyStories-33M'
#default_config['method']['gen_kwargs']['max_new_tokens'] = 46
config = TRLConfig.update(default_config, {})
print(config)

{
    "method": {
        "name": "PPOConfig",
        "ppo_epochs": 4,
        "num_rollouts": 128,
        "chunk_size": 128,
        "init_kl_coef": 0.001,
        "target": null,
        "horizon": 10000,
        "gamma": 1,
        "lam": 0.95,
        "cliprange": 0.2,
        "cliprange_value": 0.2,
        "vf_coef": 1,
        "scale_reward": "ignored",
        "ref_mean": null,
        "ref_std": null,
        "cliprange_reward": 10,
        "gen_kwargs": {
            "max_new_tokens": 40,
            "top_k": 0,
            "top_p": 1.0,
            "do_sample": true
        },
        "gen_experience_kwargs": null
    },
    "model": {
        "model_path": "roneneldan/TinyStories-33M",
        "model_arch_type": "causal",
        "num_layers_unfrozen": 2,
        "delta_kwargs": null
    },
    "optimizer": {
        "name": "adamw",
        "kwargs": {
            "lr": 3e-05,
            "betas": [
                0.9,
                0.95
            ],
            "ep

In [ ]:
def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
  sentiments = list(map(get_positive_score, sentiment_fn(samples)))
  return {"sentiments": sentiments}

def reward_fn(input: Optional[Callable[[List[str], List[str], List[str]], List[float]]]):
  

In [ ]:
# config, reward_fn, samples, eval_prompts, metric_fn, 
# prompts?
trlx.train(config=config,
           reward_fn=reward_fn)